<a href="https://colab.research.google.com/github/LxYuan0420/eat_tensorflow2_in_30_days/blob/master/notebooks/2_3_Automatic_Differentiate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [14]:
%cd "/gdrive/MyDrive/Colab Notebooks/git/eat_tensorflow2_in_30_days/notebooks"

/gdrive/MyDrive/Colab Notebooks/git/eat_tensorflow2_in_30_days/notebooks


**2-3 Automatic Differentiate**
---

The neural networks relies on back propagations to calculate gradients and update the parameters in the network. Gradient calculation is complicated which is easy to incur mistakes.

The framework of deeplearning helps us to calculate gradient automatically.

`tf.GradientTape` is usually used to record forward calculation in Tensorflow, and reverse this "tape" to obtain the gradient.

This is the automatic differentiate in TensorFlow.

**1. Calculate the Derivative Using the Gradient Tape**


In [15]:
import tensorflow as tf
import numpy as np

In [17]:
x = tf.Variable(0.0, name='x', dtype=tf.float32)
a = tf.constant(1.0)
b = tf.constant(-2.0)
c = tf.constant(1.0)

with tf.GradientTape() as tape:
    y = a*tf.pow(x, 2) + b*x + c

dy_dx = tape.gradient(y, x)
print(dy_dx)

tf.Tensor(-2.0, shape=(), dtype=float32)


In [19]:
# use watch to calculate derivatives of the constant tensor

with tf.GradientTape() as tape:
    tape.watch([a,b,c])
    y = a*tf.pow(x, 2) + b*x + c

dy_dx, dy_da, dy_db, dy_dc = tape.gradient(y, [x,a,b,c])
print(dy_dx)
print(dy_da)
print(dy_db)
print(dy_dc)

tf.Tensor(-2.0, shape=(), dtype=float32)
tf.Tensor(0.0, shape=(), dtype=float32)
tf.Tensor(0.0, shape=(), dtype=float32)
tf.Tensor(1.0, shape=(), dtype=float32)


In [21]:
# use it in the autograph

@tf.function
def f(x):
    a = tf.constant(1.0)
    b = tf.constant(-2.0)
    c = tf.constant(1.0)

    x = tf.cast(x, tf.float32)
    with tf.GradientTape() as tape:
        tape.watch(x)
        y = a*tf.pow(x,2)+b*x+c
    
    dy_dx = tape.gradient(y, x)

    return((dy_dx, y))

tf.print(f(tf.constant(0.0)))
tf.print(f(tf.constant(1.0)))

(-2, 1)
(0, 0)


**2. Calculate the Minimal Value Through the Gradient Tape and the Optimizer**

In [22]:
#Calculate the minimal value of f(x) = a*x**2 + b*x + c
#Use optimizer.minimize
#this optmizer.minimize is identical to calculating gradient using tape, then call apply_gradient

x = tf.Variable(0.0, name='x', dtype=tf.float32)

# Note that f() has no argument
def f():
    a = tf.constant(1.0)
    b = tf.constant(-2.0)
    c = tf.constant(1.0)
    y = a*tf.pow(x, 2) + b*x + c

    return (y)


optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)

for _ in range(1000):
    optimizer.minimize(f, [x])

tf.print("y =", f(),"; x =",x)

y = 0 ; x = 0.99999851


In [27]:
#Calculate minimal value in Autograph
#Use optimizer.apply_gradients

x = tf.Variable(0.0, name='x', dtype=tf.float32)
optimizer = tf.keras.optimizers.SGD(learning_rate=0.05)


# Note that f() has no argument
@tf.function
def minimizef():
    a = tf.constant(1.0)
    b = tf.constant(-2.0)
    c = tf.constant(1.0)

    for _ in tf.range(1000): #use tf.range(1000) when using autograph
        with tf.GradientTape() as tape:
            y = a*tf.pow(x, 2) + b*x + c
        dy_dx = tape.gradient(y, x)
        optimizer.apply_gradients([(dy_dx, x)])
    
    y = a*tf.pow(x, 2) + b*x + c
    return (y)

tf.print(minimizef()) 
tf.print(x)

0
0.999999762


In [28]:
# calculate minimal value in Autograph
# Use optimizer.minimize

x = tf.Variable(0.0, name='x', dtype=tf.float32)
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)


# Note that f() has no argument
@tf.function
def f():
    a = tf.constant(1.0)
    b = tf.constant(-2.0)
    c = tf.constant(1.0)

    y = a*tf.pow(x, 2) + b*x + c
    return (y)

@tf.function
def train(epoch):
    for _ in tf.range(epoch):
        optimizer.minimize(f,[x])
    return (f())

tf.print(train(1000))
tf.print(x)

0
0.99999851
